In [8]:
import numpy as np
from math import inf

def is_stable(sequence):
    #Checks if a sequence approaches a non-zero, non-infinite limit.
    if len(sequence) < 2: return False
    if 0 in sequence or inf in sequence: return False

    # Calculate differences between successive calculation steps [cite: 78]
    diffs = [abs(sequence[i] - sequence[i-1]) for i in range(1, len(sequence))]
    avg_diff = sum(diffs) / len(diffs) 

    for i in range(1, len(diffs)):
        # If the gap between steps grows and is larger than average, 
        # it hasn't converged
        if diffs[i] > diffs[i-1] and diffs[i] > avg_diff:
            return False
    return True

def classify_dt_signal(signal_func):
    energies = []
    powers = []
    
    # Test for increasing 'N' (Number of samples)
    # We check the signal from N=100 to N=1,000,000
    for i in range(2, 7):
        N = 10**i
        n = np.arange(-N, N)
        x_n = signal_func(n)
        
        # 1. Compute Signal Energy: Sum of squares
        E = np.sum(np.abs(x_n)**2)
        
        # 2. Compute Average Power: Energy / Total Duration 
        P = E / len(n)
        
        energies.append(E)
        powers.append(P)

    # Classification Logic
    if is_stable(energies):
        return f"Energy Signal (E ≈ {energies[-1]:.4f})"
    elif is_stable(powers):
        return f"Power Signal (P ≈ {powers[-1]:.4f})"
    else:
        return "Neither Energy nor Power Signal"

# --- Examples ---
# 1. Rectangular Pulse (Energy Signal)
def rect(n): return np.where((n >= 0) & (n <= 10), 1, 0)

# 2. Unit Step (Power Signal)
def unit_step(n): return np.where(n >= 0, 1, 0)
    
def dc_signal(n):
    # Constant value of 2 for all indices
    return np.full_like(n, 2, dtype=float)

'''
def growing_exp(n):
    # Returns e^n for n >= 0
    # Note: We cap n to avoid float overflow errors
    return np.where((n >= 0) & (n < 20), np.exp(n), 0)
'''
def growing_exp(n):
    try:
        # A slower growth like 1.1^n helps avoid instant overflow 
        # so you can actually see the instability.
        val = 1.1**n 
        return np.where(n >= 0, val, 0)
    except:
        return float('inf')

print(f"Rect Pulse: {classify_dt_signal(rect)}")
print(f"Unit Step: {classify_dt_signal(unit_step)}")
print(f"sine wave: {classify_dt_signal(np.sin)}")
print(f"dc/constant signal: {classify_dt_signal(dc_signal)}")
print(f"growing exponential signal: {classify_dt_signal(growing_exp)}")

Rect Pulse: Energy Signal (E ≈ 11.0000)
Unit Step: Power Signal (P ≈ 0.5000)
sine wave: Power Signal (P ≈ 0.5000)
dc/constant signal: Power Signal (P ≈ 4.0000)
growing exponential signal: Neither Energy nor Power Signal


/tmp/ipykernel_5874/218192668.py:69: RuntimeWarning: overflow encountered in power
  val = 1.1**n
/tmp/ipykernel_5874/218192668.py:32: RuntimeWarning: overflow encountered in square
  E = np.sum(np.abs(x_n)**2)
/home/aaron/Documents/signal_classifier/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
